In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('once')

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import units as u
from astropy import coordinates, cosmology
from astroquery.simbad import Simbad
Simbad.add_votable_fields('z_value')

from ekfphys import observer, calibrations
from ekfplot import plot as ek
from ekfparse import query

from agrias import utils
from skipper import observe

import sys
sys.path.append('../scripts')
import reader

cosmo = cosmology.FlatLambdaCDM(70.,0.3)

In [3]:
ms = reader.merianselect()

/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/astropy/cosmology/flrw/base.py:1351: RuntimeWarning: divide by zero encountered in log10
  val = 5.0 * np.log10(abs(self.luminosity_distance(z).value)) + 25.0
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1

[merianselect] Only choosing sources with m_i < 28.0
[merianselect] only choosing sources with cmodel/gaap(i) > 1.3
[merianselect] only choosing sources with griz photometry


In [4]:
lha_cat = calibrations.SFR2LHa( 10.**(ms['logssfr']+ ms['logmass']))
fha_cat = lha_cat / (4.*np.pi * cosmo.luminosity_distance(ms['z500'].values).to(u.cm).value**2)
ms['haew_expected'] = calibrations.EWfromLHa(lha_cat, ms['Mr'])

In [5]:
mcoords = coordinates.SkyCoord(ms['RA'].values, ms['DEC'].values, unit='deg')

In [6]:
Nn = 10
neighbor_indices = np.zeros([len(mcoords), Nn-1], dtype=int)
neighbor_distances = np.zeros([len(mcoords), Nn-1], dtype=float)

for nthneighbor in range(2, Nn+1):
    idx, d2d,_ = mcoords.match_to_catalog_sky(mcoords, nthneighbor=nthneighbor)
    #is_nearby = (cosmo.kpc_proper_per_arcmin(0.08) * d2d) < (200.*u.kpc)
    neighbor_distances[:,nthneighbor-2] = (cosmo.kpc_proper_per_arcmin(0.08) * d2d).to(u.kpc)
    neighbor_indices[:,nthneighbor-2]   = idx

In [7]:
is_nearby = (neighbor_distances>10.)&(neighbor_distances<100.)

In [11]:
has_neighbors.sum()

8386

In [ ]:
has_neighbors = is_nearby.sum(axis=1) > 1
colnames = ['RA','DEC','logmass','z500','N708_mag', 'r_mag', 'haew_expected']
for band in ['r','N708']:
    ms[f'{band}_mag'] = utils.flux2mag(ms[f'{band}_cModelFlux_Merian'])
is_incat = pd.Series(index=ms.index, dtype=bool)
is_incat.loc[:] = False
nsaved=0
for midx in range(has_neighbors.sum()):
    df = ms.loc[has_neighbors].iloc[[midx]].copy()[colnames]
     
    indices = neighbor_indices[has_neighbors][midx]
    distances = neighbor_distances[has_neighbors][midx]
    indices = indices[distances<200.]
    
    neighbors = ms.loc[ms.index[indices], colnames]
    df = pd.concat([df,neighbors])

    df = df.loc[~df['logmass'].isna()]
    if df.shape[0] < 2:
        continue
    elif (df['logmass'] > 9.).any():
        continue    
    elif is_incat.loc[df.index].any():        
        continue

        
    matches = Simbad.query_region( coordinates.SkyCoord(df.RA.values, df.DEC.values, unit='deg'), radius=10.*u.arcsec)
    if matches is not None:
        zlit_matches = df.index[matches['SCRIPT_NUMBER_ID']-1]
        df.loc[zlit_matches,'z_lit'] = matches['Z_VALUE']
        out_of_band = (df['z_lit']<0.05)|(df['z_lit']>0.12)
        df = df.loc[~out_of_band]
        
    is_incat.loc[df.index] = True
    
    if df.shape[0] < 2:
        continue
        
    ncoords = coordinates.SkyCoord(df['RA'].values,df['DEC'].values, unit='deg')
    
    is_duplicate = np.zeros(len(df), dtype=bool)
    for nx in range(len(df.index)):
        nd2d = ncoords[nx].separation(ncoords).to(u.arcsec)
        is_notself = nd2d>0.

        if (nd2d[is_notself].min() > (10.*u.arcsec)) or is_duplicate[nx]:
            continue        
            
        is_duplicate[(nd2d>0.)&(nd2d<10.*u.arcsec)] = True
        
    df = df.loc[~is_duplicate]
    if df.shape[0] <= 3:
        continue
    df.to_csv(f'~/Desktop/potential_groups/{df.index[0]}.csv', index_label='NAME')
    
    nsaved+=1


/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '2:18:26.09172116 -6:18:21.88643484': No astronomical object found :  
  warnings.warn("Warning: The script line number %i raised "
/Users/kadofong/opt/miniconda3/envs/merian/lib/python3.10/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '2:21:5

In [9]:
df

,RA,DEC,logmass,z500,N708_mag,r_mag,haew_expected
M2951014442160118786,35.461983,-6.388296,8.478662,0.074997,20.682032,20.783830,34.388787
M2951014442160118012,35.468951,-6.385358,7.626475,0.085343,21.859561,22.477184,159.816922
M2951014442160118008,35.472728,-6.391249,8.513450,0.072329,19.794949,19.950256,16.131905
M2951014442160117883,35.441696,-6.392504,8.477709,0.081521,20.654035,20.919890,58.923178


In [297]:
out_of_band

M2950345939090448953    False
M2950385521509030799    False
M2950345939090450206    False
Name: z_lit, dtype: bool

In [ ]:
glob.glob('~/Desktop/

In [266]:
ncoords[2].separation(ncoords).to(u.arcsec)

<Angle [  6.62971942,   6.36975812,   0.        , 191.28366649,
        192.20208773, 104.42510414] arcsec>

In [244]:
nd2d.to(u.arcsec)

<Angle [ 3.20103336,  3.20103336,  6.36975812, 13.67178007, 13.67178007,
        99.72026021] arcsec>

In [243]:
df

,RA,DEC,logmass,z500,r_mag,z_lit
NAME,,,,,,
M3327408058673439689,168.299075,-1.030943,8.020094,0.081046,21.171409,NaN
M3327408058673437965,168.272082,-1.037184,8.595204,0.080025,20.844713,NaN
M3327408058673437108,168.324718,-1.044508,NaN,0.082224,20.954357,0.06834
M3327408058673437967,168.271440,-1.040927,8.094446,0.084173,21.465407,NaN
M3327408058673437104,168.326492,-1.044013,NaN,0.086028,17.845780,0.06834
M3327408058673437107,168.326444,-1.044901,NaN,0.066046,19.915117,0.06834
